# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.widgets import RunDetails
from pprint import pprint
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import requests

/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook134576/code/Users/odl_user_134576/nd00333-capstone-master/train.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_red["wine_type"] = "RED"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'wine-quality-automl'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "wine-quality"
description_text = "Wine Quality Dataset for Udacity Course 3"

if key in ws.datasets.keys(): 
    found = True
    input_data = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    url_white = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
    url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    white_data = TabularDatasetFactory.from_delimited_files(url_white, separator=";")
    red_data = TabularDatasetFactory.from_delimited_files(url_red, separator=";")
    features, target = clean_data(white_data, red_data, numeric_target=False)
    features["quality"]=target
    ds = ws.get_default_datastore()
    input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=features, target=ds, name=key,
                                                             description=description_text)


df = input_data.to_pandas_dataframe()
df.describe()


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A47KBR6Z4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/fc284e72-2f53-4fd5-adb7-e121d48c0c99/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is_red_wine
count,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00,6491.00
mean,7.22,0.34,0.32,5.44,0.06,30.52,115.71,0.99,3.22,0.53,10.49,0.25
std,1.30,0.16,0.15,4.76,0.03,17.75,56.46,0.00,0.16,0.15,1.19,0.43
min,3.80,0.08,0.00,0.60,0.01,1.00,6.00,0.99,2.72,0.22,8.00,0.00
25%,6.40,0.23,0.25,1.80,0.04,17.00,77.00,0.99,3.11,0.43,9.50,0.00
50%,7.00,0.29,0.31,3.00,0.05,29.00,118.00,0.99,3.21,0.51,10.30,0.00
75%,7.70,0.40,0.39,8.10,0.07,41.00,156.00,1.00,3.32,0.60,11.30,0.00
max,15.90,1.58,1.66,65.80,0.61,289.00,440.00,1.04,4.01,2.00,14.90,1.00


In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
automl_settings = {
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "primary_metric": "AUC_weighted",
    "max_concurrent_iterations": 4,
    
}
automl_config = AutoMLConfig(
    experiment_timeout_hours=1.0,
    task="classification",
    compute_target=compute_target,
    training_data=input_data,
    label_column_name="quality",
    **automl_settings)

In [11]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, tag={'type':'automl'}, show_output=True)

Running on remote.
No run_configuration provided, running on aml-compute with default configuration
Running on remote compute: aml-compute
Parent Run ID: AutoML_cfdf3dde-5e87-4512-886c-70b176ccd8df

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------------

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
automl_run.wait_for_completion()
assert (automl_run.get_status() == "Completed")
best_ml_run, best_model = automl_run.get_output()


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators':
                       list(e[0] for e in step[1].estimators),
                   'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(best_model)


datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['35', '32', '33', '37', '39', '48', '47', '20'],
 'weights': [0.4,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.2,
             0.06666666666666667]}

35 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

35 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'eta': 0.5,
 'gamma': 0.1,
 'le

In [14]:
best_ml_run.download_file("./outputs/model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [48]:
from azureml.core.conda_dependencies import CondaDependencies
registered_model = best_ml_run.register_model(model_name="wine-quality-model", model_path="./outputs/model.pkl")

myenv = CondaDependencies()
for conda_pkg in ["scikit-learn==0.24.0", "py-xgboost==0.90","numpy==1.19.5"]:
    myenv.add_conda_package(conda_pkg)

for pip_pkg in ["azureml-defaults==1.20.0","azureml-automl-core==1.20.0","azureml-automl-runtime==1.20.0","packaging==20.8"]:
    myenv.add_pip_package(pip_pkg)


with open("env.yml","w") as f:
    f.write(myenv.serialize_to_string())

service_name = 'wine-quality-classification'
inference_config = InferenceConfig(entry_script='score.py', conda_file="env.yml")
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.get_logs)
print(service.scoring_uri)
primary_key, _ = service.get_keys()

WebserviceException: WebserviceException:
	Message: Conflict of operation, another operation on same entity is already running in workspace quick-starts-ws-134576.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Conflict of operation, another operation on same entity is already running in workspace quick-starts-ws-134576."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {"data":
         [{
             "is_red_wine": 1,
             "fixed_acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000
         },
         {
             "is_red_wine": 0,
             "fixed_acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000
         }]
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri, json_data, headers=header)
print(resp)

TODO: In the cell below, print the logs of the web service and delete the service

In [47]:
print(service.get_logs())
#service.delete()

ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 13 Jan 2021 20:20:06 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': 'ebb4bf7adffd4cca8278dd8b53a6f53c', 'x-ms-client-session-id': 'aea6cdf1-e655-44f6-9fe6-026d6f82eca7', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.119', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/wine-quality-classification-3xj5XPh1wE_xHT7gxkN_dw\' under resource group \'aml-quickstarts-134576\' was not found. For more details please go to 

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 13 Jan 2021 20:20:06 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': 'ebb4bf7adffd4cca8278dd8b53a6f53c', 'x-ms-client-session-id': 'aea6cdf1-e655-44f6-9fe6-026d6f82eca7', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.119', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/wine-quality-classification-3xj5XPh1wE_xHT7gxkN_dw\' under resource group \'aml-quickstarts-134576\' was not found. For more details pl